In [1]:
import datetime
import pickle
from urllib.request import urlopen
import os
import pandas as pd
import numpy as np
import gensim
import pickle
import matplotlib.pyplot as plt

import heapq

In [13]:
from pyspark.ml.clustering import LDA
from pyspark.ml.feature import CountVectorizer
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("SparkSession") \
    .getOrCreate()

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Galeforse/DST-Assessment-05/main/Data/NCSC%20Reports.csv')

In [3]:
import nltk
from nltk.corpus import stopwords

In [4]:
l1 = stopwords.words('english')
l2 = ['have', 'with', 'cannot', 'all', 'more', 'less', 'fewer', 'if', 'from', 'new', 'old', 'be', 'This']
l1.append(l2)
lst1 = [' {} '.format(x) for x in l1]

len(lst1)

180

In [5]:
df['Article'].replace(lst1, " ", regex=True, inplace=True)
df['Article'].replace(r'[^\w\s]', " ", regex=True, inplace=True)
df['Article'].replace('[0-9999999999]', " ", regex=True, inplace=True)

In [9]:
df=df.drop(df.columns[0],axis=1)

In [11]:
df=df.drop("Links",axis=1)

In [12]:
df.head()

,Title,Article,topics
0,23rd April 2021,The NCSC aware malicious piece spyware kno...,"['Cyber attack', 'Cyber strategy', 'Education'..."
1,16th April 2021,Cyber security researchers uncovered series ...,"['Cyber strategy', 'Patching', 'Vulnerabilities']"
2,12th April 2021,Cyber security researchers Esentire warned...,"['Phishing', 'Social media', 'Personal data', ..."
3,2nd April 2021,The UK education sector continues face incre...,"['Education', 'Incident management', 'Secure d..."
4,26th March 2021,Earlier month Microsoft confirmed sophistica...,"['Cyber attack', 'Education', 'Mitigation', 'P..."


In [16]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [19]:
df_lem = df['Article'].apply(lemmatize_text)
df_lem_list = df_lem.values.tolist()

In [23]:
lister = [[] for x in range(len(df_lem))]
for i in range(len(df_lem_list)):
    l = df_lem_list[i]
    for token in l:
        if (len(token)>4 or token.isupper()):
            lister[i].append(token)

In [27]:
len(lister)

221

In [28]:
type(lister[0])

list

In [31]:
dftemp = spark.createDataFrame([(df["Title"][0],lister[0])],["label","raw"])

In [33]:
for i in range(1,221):
    newrow = spark.createDataFrame([(df["Title"][i],lister[i])],["label","raw"])
    dftemp = dftemp.union(newrow)

In [36]:
dftemp.head(2)

[Row(label='23rd April 2021', raw=['NCSC', 'aware', 'malicious', 'piece', 'spyware', 'known', 'FluBot', 'affecting', 'Android', 'phone', 'device', 'across', 'UK', 'spyware', 'installed', 'victim', 'receives', 'message', 'asking', 'install', 'tracking', 'missed', 'package', 'delivery', 'nScammers', 'cyber', 'criminal', 'regularly', 'exploit', 'known', 'trusted', 'brand', 'personal', 'FluBot', 'campaign', 'prime', 'example', 'nAndroid', 'urged', 'familiarise', 'guidance', 'vigilant', 'suspicious', 'looking', 'message', 'forwarded', 'Earlier', 'FireEye', 'published', 'saying', 'APT', 'actor', 'actively', 'exploiting', 'vulnerability', 'Pulse', 'Connect', 'VPN', 'appliance', 'follows', 'security', 'update', 'Pulse', 'Secure', 'recently', 'discovered', 'limited', 'number', 'customer', 'experienced', 'evidence', 'exploit', 'behaviour', 'Pulse', 'Connect', 'Secure', 'PCS', 'appliance', 'NCSC', 'aware', 'unauthenticated', 'remote', 'execution', 'vulnerability', 'affecting', 'Pulse', 'Connect',

In [ ]:
cv = CountVectorizer()
cv.setInputCol("raw")
cv.setOutputCol("vectors")
df2 = cv.fit(dftemp)
df2.setInputCol("raw")
df2.transform(dftemp)

In [15]:
df["Title"][0]

'23rd April 2021'